<a href="https://colab.research.google.com/github/adigenova/uohpmd/blob/main/code/NoSQL_ArangoDB_ejemplo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![arangodb](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/img/ArangoDB_logo.png?raw=1)

# Tutorial de  Arango DB

ArangoDB es una base de datos multimodelo  con modelos de datos flexibles para documentos, grafos y clave/valor. Posee el lenguaje AQL que es similar al SQL para realizar consultas.

# Instalación 

Antes de comenzar con ArangoDB, debemos preparar nuestro entorno y crear una base de datos temporal en el Servicio Oasis.

In [ ]:
%%capture
!git clone -b oasis_connector --single-branch https://github.com/arangodb/interactive_tutorials.git
!rsync -av interactive_tutorials/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [ ]:
import json
import requests
import sys
import oasis
import time

from pyArango.connection import *
from arango import ArangoClient

Creamos una base de datos temporal

In [ ]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="AQLCrudTutorial", credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')
# Connect to the temp database
conn = oasis.connect(login)
pyAr_db = conn[login["dbName"]]

Requesting new temp credentials.
Temp database ready to use.


In [ ]:
python_arango_db = oasis.connect_python_arango(login)
aql = python_arango_db.aql

In [ ]:
print("https://{}:{}".format(login["hostname"], login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

https://tutorials.arangodb.cloud:8529
Username: TUTyldq5mspzo9jvsgn79708
Password: TUTe56xixvmuxchib9ht1jdrv
Database: TUTy29y9plwkarpz9mkn2c628


Podemos usar la URL anterior para revisar nuestra base de datos arango temporal.



## Creando Colecciones

Antes de que podamos insertar documentos con AQL, necesitamos un lugar para colocarlos: una colección

In [ ]:
# creamos la coleccion usando pyArango
pyAr_db.createCollection(name="Personajes")
if python_arango_db.has_collection("Personajes"):
  pass
else:
  python_arango_db.create_collection("Pesonajes")

##  Creando y leyendo documentos

La sintaxis para crear un nuevo documento es INSERT document INTO collectionName. El documento es un objeto JSON, que se compone de pares de clave de atributo y valor.

Las claves son siempre secuencias de caracteres (cadenas), mientras que los valores de los atributos pueden tener diferentes tipos:

* null
* boolean (true, false)
* number (integer and floating point)
* string
* array
* object


In [ ]:
insert_query = """
INSERT {
    "nombre": "Alex",
    "apellido": "Di Genova",
    "vivo": true,
    "edad": 41,
    "caracteres": ["A","H","C","N","P"]
} INTO Personajes
"""
# python-arango
aql.execute(insert_query)

<Cursor>

Chequamos si la inserción se realizo con exito en la colleción Personajes.
Para cada documento de la colección, a c se le asigna un documento, que luego se devuelve según el cuerpo del bucle.

In [ ]:
todos_personajes = """
FOR c IN Personajes
    RETURN c
"""
query_result = aql.execute(todos_personajes) 
for doc in  query_result:
 print(doc)

{'_key': '4168077885', '_id': 'Personajes/4168077885', '_rev': '_edAjaQK---', 'apellido': 'Di Genova', 'caracteres': ['A', 'H', 'C', 'N', 'P'], 'edad': 41, 'nombre': 'Alex', 'vivo': True}


In [ ]:
insert_query = """
LET data = [
    { "nombre": "Roberto", "apellido": "Ramirez", "vivo": false, "caracteres": ["A","H","C"] },
    { "nombre": "Jaime", "apellido": "Quinteros", "vivo": true, "edad": 36, "caracteres": ["A","F","B"] },
    { "nombre": "Catalina", "apellido": "rojas", "vivo": false, "edad": 40, "caracteres": ["D","H","C"] },
    { "nombre": "Cesar", "apellido": "Lanaro", "vivo": true, "edad": 36, "caracteres": ["H","E","F"] },
    { "nombre": "David", "apellido": "Tardon", "vivo": true, "edad": 16, "caracteres": ["D","H","C"] },
    { "nombre": "Jose", "apellido": "Mora", "vivo": false, "caracteres": ["A","B","C","F"] },
    { "nombre": "Pedro", "apellido": "Baesa", "vivo": false, "caracteres": ["E","G","F"] },
    { "nombre": "Guillermo", "apellido": "Toro", "vivo": false, "caracteres": ["O","L","N"] },
    { "nombre": "Juan", "apellido": "Niebla", "vivo": true, "edad": 16, "caracteres": ["A","B","C","F"] },
    { "nombre": "Sara", "apellido": "Rojas", "vivo": true, "edad": 13, "caracteres": ["D","I","J"] },
    { "nombre": "Ariela", "apellido": "Araya", "vivo": true, "edad": 11, "caracteres": ["C","K","L"] },
    { "nombre": "Roberto", "apellido": "Cabezas", "vivo": false, "caracteres": ["A","B","C","K"] },
    { "nombre": "Theon", "apellido": "Greyjoy", "vivo": true, "edad": 16, "caracteres": ["E","R","K"] },
    { "nombre": "Bran", "apellido": "Stark", "vivo": true, "edad": 10, "caracteres": ["L","J"] },
    { "nombre": "Joffrey", "apellido": "Baratheon", "vivo": false, "edad": 19, "caracteres": ["I","L","O"] },
    { "nombre": "Sandor", "apellido": "Clegane", "vivo": true, "caracteres": ["A","P","K","F"] },
    { "nombre": "Tyrion", "apellido": "Lannister", "vivo": true, "edad": 32, "caracteres": ["F","K","M","N"] },
    { "nombre": "Khal", "apellido": "Drogo", "vivo": false, "caracteres": ["A","C","O","P"] },
    { "nombre": "Tywin", "apellido": "Lannister", "vivo": false, "caracteres": ["O","M","H","F"] },
    { "nombre": "Davos", "apellido": "Seaworth", "vivo": true, "edad": 49, "caracteres": ["C","K","P","F"] },
    { "nombre": "Samwell", "apellido": "Tarly", "vivo": true, "edad": 17, "caracteres": ["C","L","I"] },
    { "nombre": "Stannis", "apellido": "Baratheon", "vivo": false, "caracteres": ["H","O","P","M"] },
    { "nombre": "Melisandre", "vivo": true, "caracteres": ["G","E","H"] },
    { "nombre": "Margaery", "apellido": "Tyrell", "vivo": false, "caracteres": ["M","D","B"] },
    { "nombre": "Jeor", "apellido": "Mormont", "vivo": false, "caracteres": ["C","H","M","P"] },
    { "nombre": "Bronn", "vivo": true, "caracteres": ["K","E","C"] },
    { "nombre": "Varys", "vivo": true, "caracteres": ["M","F","N","E"] },
    { "nombre": "Shae", "vivo": false, "caracteres": ["M","D","G"] },
    { "nombre": "Talisa", "apellido": "Maegyr", "vivo": false, "caracteres": ["D","C","B"] },
    { "nombre": "Gendry", "vivo": false, "caracteres": ["K","C","A"] },
    { "nombre": "Ygritte", "vivo": false, "caracteres": ["A","P","K"] },
    { "nombre": "Tormund", "apellido": "Giantsbane", "vivo": true, "caracteres": ["C","P","A","I"] },
    { "nombre": "Gilly", "vivo": true, "caracteres": ["L","J"] },
    { "nombre": "Brienne", "apellido": "Tarth", "vivo": true, "edad": 32, "caracteres": ["P","C","A","K"] },
    { "nombre": "Ramsay", "apellido": "Bolton", "vivo": true, "caracteres": ["E","O","G","A"] },
    { "nombre": "Ellaria", "apellido": "Sand", "vivo": true, "caracteres": ["P","O","A","E"] },
    { "nombre": "Daario", "apellido": "Naharis", "vivo": true, "caracteres": ["K","P","A"] },
    { "nombre": "Missandei", "vivo": true, "caracteres": ["D","L","C","M"] },
    { "nombre": "Tommen", "apellido": "Baratheon", "vivo": true, "caracteres": ["I","L","B"] },
    { "nombre": "Jaqen", "apellido": "H'ghar", "vivo": true, "caracteres": ["H","F","K"] },
    { "nombre": "Roose", "apellido": "Bolton", "vivo": true, "caracteres": ["H","E","F","A"] },
    { "nombre": "The High Sparrow", "vivo": true, "caracteres": ["H","M","F","O"] }
]

FOR d IN data
    INSERT d INTO Personajes
"""
# python-arango
aql.execute(insert_query)

<Cursor>

In [ ]:
todos_personajes_nombre = """
FOR c IN Personajes
    RETURN { n:c.nombre,e:c.edad, v:c.vivo }
"""
# python-arango
query_result = aql.execute(todos_personajes_nombre)
for doc in  query_result:
  print(doc)

{'n': 'Alex', 'e': 41, 'v': True}
{'n': 'Roberto', 'e': None, 'v': False}
{'n': 'Jaime', 'e': 36, 'v': True}
{'n': 'Catalina', 'e': 40, 'v': False}
{'n': 'Cesar', 'e': 36, 'v': True}
{'n': 'David', 'e': 16, 'v': True}
{'n': 'Jose', 'e': None, 'v': False}
{'n': 'Pedro', 'e': None, 'v': False}
{'n': 'Guillermo', 'e': None, 'v': False}
{'n': 'Juan', 'e': 16, 'v': True}
{'n': 'Sara', 'e': 13, 'v': True}
{'n': 'Ariela', 'e': 11, 'v': True}
{'n': 'Roberto', 'e': None, 'v': False}
{'n': 'Theon', 'e': 16, 'v': True}
{'n': 'Bran', 'e': 10, 'v': True}
{'n': 'Joffrey', 'e': 19, 'v': False}
{'n': 'Sandor', 'e': None, 'v': True}
{'n': 'Tyrion', 'e': 32, 'v': True}
{'n': 'Khal', 'e': None, 'v': False}
{'n': 'Tywin', 'e': None, 'v': False}
{'n': 'Davos', 'e': 49, 'v': True}
{'n': 'Samwell', 'e': 17, 'v': True}
{'n': 'Stannis', 'e': None, 'v': False}
{'n': 'Melisandre', 'e': None, 'v': True}
{'n': 'Margaery', 'e': None, 'v': False}
{'n': 'Jeor', 'e': None, 'v': False}
{'n': 'Bronn', 'e': None, 'v': Tr

# Actualizando Documentos

In [ ]:
revivir_todos_personajes= """
FOR c IN Personajes
    FILTER c.vivo == False
    RETURN c._key
"""

neds_document_key = list()

# python-arango
query_result = aql.execute(revivir_todos_personajes)

for doc in  query_result:
    print("_key: " + str(doc))
    neds_document_key.append(doc)

_key: 266302775424
_key: 266302775426
_key: 266302775429
_key: 266302775430
_key: 266302775431
_key: 266302775435
_key: 266302775438
_key: 266302775441
_key: 266302775442
_key: 266302775445
_key: 266302775447
_key: 266302775448
_key: 266302775451
_key: 266302775452
_key: 266302775453


In [ ]:
neds_document_key[0]

'266302775424'



actualizando un documento por key:

In [ ]:
muertos = """
UPDATE @key 
WITH { vivo: True} 
IN Personajes
"""
bindVars = {'key': neds_document_key[0]}

aql.execute(muertos, batch_size=1, bind_vars=bindVars)

todos_personajes = """
FOR c IN Personajes
  RETURN { k:c._key, n:c.nombre,e:c.edad, v:c.vivo }
"""
query_result = aql.execute(todos_personajes)
for doc in  query_result:
    print(doc)

{'k': '4168077885', 'n': 'Alex', 'e': 41, 'v': True}
{'k': '266302775425', 'n': 'Jaime', 'e': 36, 'v': True}
{'k': '266302775426', 'n': 'Catalina', 'e': 40, 'v': False}
{'k': '266302775427', 'n': 'Cesar', 'e': 36, 'v': True}
{'k': '266302775428', 'n': 'David', 'e': 16, 'v': True}
{'k': '266302775429', 'n': 'Jose', 'e': None, 'v': False}
{'k': '266302775430', 'n': 'Pedro', 'e': None, 'v': False}
{'k': '266302775431', 'n': 'Guillermo', 'e': None, 'v': False}
{'k': '266302775432', 'n': 'Juan', 'e': 16, 'v': True}
{'k': '266302775433', 'n': 'Sara', 'e': 13, 'v': True}
{'k': '266302775434', 'n': 'Ariela', 'e': 11, 'v': True}
{'k': '266302775435', 'n': 'Roberto', 'e': None, 'v': False}
{'k': '266302775436', 'n': 'Theon', 'e': 16, 'v': True}
{'k': '266302775437', 'n': 'Bran', 'e': 10, 'v': True}
{'k': '266302775438', 'n': 'Joffrey', 'e': 19, 'v': False}
{'k': '266302775439', 'n': 'Sandor', 'e': None, 'v': True}
{'k': '266302775440', 'n': 'Tyrion', 'e': 32, 'v': True}
{'k': '266302775441', 'n'

# Usando `FOR` para actualizar todos los personajes:

In [ ]:
actualizar_equipo_vivo = """
FOR c IN Personajes
    UPDATE c WITH { vivo: True, equipo : "Colo Colo" } IN Personajes
"""
# python-arango
aql.execute(actualizar_equipo_vivo)

todos_personajes_equipo_vivo = """
FOR c IN Personajes
    RETURN {"Nombre" : c.nombre, "Equipo" : c.equipo, "Vivo": c.vivo}
"""
# python-arango
query_result = aql.execute(todos_personajes_equipo_vivo)

for doc in  query_result:
    print(doc)

{'Nombre': 'Roberto', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Jaime', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Catalina', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Cesar', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'David', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Jose', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Pedro', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Guillermo', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Juan', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Sara', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Ariela', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Roberto', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Theon', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Bran', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Joffrey', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Sandor', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Tyrion', 'Equipo': 'Colo Colo', 'Vivo': True}
{'Nombre': 'Khal', 

# Borrando Documentos por clave

Para eliminar documentos de una coleccion podemos utilizar REMOVE

In [ ]:
buscando_alex= """
FOR c IN Personajes
    FILTER c.nombre == "Alex"
    RETURN c._key
"""
# python-arango
query_result = aql.execute(buscando_alex)

id_alex = None
for doc in  query_result:
    print(doc)
    id_alex=doc

4168077885


In [ ]:
remove_alex = """
REMOVE @key IN Personajes
"""
bindVars = {'key': id_alex}

try:
    # python-arango
    aql.execute(remove_alex, bind_vars=bindVars)
except:
    print("Alex fue borrado.")

encontrar_alex = """
FOR c IN Personajes
    FILTER c.nombre == "Alex"
    RETURN c
"""
# python-arango
query_result = aql.execute(encontrar_alex, count=True)

if len(query_result) == 0 :
    print("Alex no fue encontrado.")

Alex fue borrado.
Alex no fue encontrado.


Con un FOR podemos eliminar todos los documentos de la coleccion:

In [ ]:
borrar_todos = """
FOR c IN Personajes
    REMOVE c IN Personajes
"""
# python-arango
aql.execute(borrar_todos)

todos_personajes = """
FOR c IN Personajes
    RETURN c
"""
# python-arango
query_result = aql.execute(todos_personajes, count=True)

if len(query_result) == 0 :
    print("Todos los personajes fueron borrados.")

In [ ]:
filtro_edad = """
FOR c IN Personajes
    FILTER c.edad < 20 AND c.edad != null
    RETURN { nombre: c.nombre, edad: c.edad }
"""
# python-arango
query_result = aql.execute(filtro_edad)

for doc in  query_result:
    print(doc)

NameError: ignored

In [ ]:
sort_nombre = """
FOR c IN Personajes
    SORT c.nombre
    RETURN c
"""
query_result = aql.execute(sort_nombre)

for doc in  query_result:
    print(doc)

{'_key': '266302775434', '_id': 'Personajes/266302775434', '_rev': '_edBIuVq--I', 'nombre': 'Ariela', 'apellido': 'Araya', 'vivo': True, 'edad': 11, 'caracteres': ['C', 'K', 'L'], 'equipo': 'Colo Colo'}
{'_key': '266302775437', '_id': 'Personajes/266302775437', '_rev': '_edBIuVu---', 'nombre': 'Bran', 'apellido': 'Stark', 'vivo': True, 'edad': 10, 'caracteres': ['L', 'J'], 'equipo': 'Colo Colo'}
{'_key': '266302775457', '_id': 'Personajes/266302775457', '_rev': '_edBIuVu--S', 'nombre': 'Brienne', 'apellido': 'Tarth', 'vivo': True, 'edad': 32, 'caracteres': ['P', 'C', 'A', 'K'], 'equipo': 'Colo Colo'}
{'_key': '266302775449', '_id': 'Personajes/266302775449', '_rev': '_edBIuVu--K', 'nombre': 'Bronn', 'vivo': True, 'caracteres': ['K', 'E', 'C'], 'equipo': 'Colo Colo'}
{'_key': '266302775426', '_id': 'Personajes/266302775426', '_rev': '_edBIuVq--A', 'nombre': 'Catalina', 'apellido': 'rojas', 'vivo': True, 'edad': 40, 'caracteres': ['D', 'H', 'C'], 'equipo': 'Colo Colo'}
{'_key': '26630277

In [ ]:
sort_nombre = """
FOR c IN Personajes
    SORT c.nombre DESC
    RETURN c
"""
query_result = aql.execute(sort_nombre)

for doc in  query_result:
    print(doc)

{'_key': '266302775454', '_id': 'Personajes/266302775454', '_rev': '_edBIuVu--P', 'nombre': 'Ygritte', 'vivo': True, 'caracteres': ['A', 'P', 'K'], 'equipo': 'Colo Colo'}
{'_key': '266302775450', '_id': 'Personajes/266302775450', '_rev': '_edBIuVu--L', 'nombre': 'Varys', 'vivo': True, 'caracteres': ['M', 'F', 'N', 'E'], 'equipo': 'Colo Colo'}
{'_key': '266302775442', '_id': 'Personajes/266302775442', '_rev': '_edBIuVu--D', 'nombre': 'Tywin', 'apellido': 'Lannister', 'vivo': True, 'caracteres': ['O', 'M', 'H', 'F'], 'equipo': 'Colo Colo'}
{'_key': '266302775440', '_id': 'Personajes/266302775440', '_rev': '_edBIuVu--B', 'nombre': 'Tyrion', 'apellido': 'Lannister', 'vivo': True, 'edad': 32, 'caracteres': ['F', 'K', 'M', 'N'], 'equipo': 'Colo Colo'}
{'_key': '266302775455', '_id': 'Personajes/266302775455', '_rev': '_edBIuVu--Q', 'nombre': 'Tormund', 'apellido': 'Giantsbane', 'vivo': True, 'caracteres': ['C', 'P', 'A', 'I'], 'equipo': 'Colo Colo'}
{'_key': '266302775462', '_id': 'Personaje

In [ ]:
sort_nombre = """
FOR c IN Personajes
    SORT c.nombre 
    LIMIT 10
    RETURN c.nombre
"""

query_result = aql.execute(sort_nombre)

for doc in  query_result:
    print(doc)

Ariela
Bran
Brienne
Bronn
Catalina
Cesar
Daario
David
Davos
Ellaria


In [ ]:
pyAr_db.createCollection(name="caracteres")

ArangoDB collection name: caracteres, id: 4165857923, type: document, status: loaded

In [ ]:
insert_query = """
LET data = [
    { "_key": "A", "en": "strong", "es": "fuerte" },
    { "_key": "B", "en": "polite", "es": "polite" },
    { "_key": "C", "en": "loyal", "es": "leal" },
    { "_key": "D", "en": "beautiful", "es": "bonito" },
    { "_key": "E", "en": "sneaky", "es": "insidioso" },
    { "_key": "F", "en": "experienced", "es": "experimentado" },
    { "_key": "G", "en": "corrupt", "es": "corrupto" },
    { "_key": "H", "en": "powerful", "es": "potente" },
    { "_key": "I", "en": "naive", "es": "ingenuo" },
    { "_key": "J", "en": "unmarried", "es": "soltero" },
    { "_key": "K", "en": "skillful", "es": "habil" },
    { "_key": "L", "en": "young", "es": "joven" },
    { "_key": "M", "en": "smart", "es": "inteligente" },
    { "_key": "N", "en": "rational", "es": "racional" },
    { "_key": "O", "en": "ruthless", "es": "implacable" },
    { "_key": "P", "en": "brave", "es": "valiente" },
    { "_key": "Q", "en": "mighty", "es": "poderoso" },
    { "_key": "R", "en": "weak", "es": "debil" }
]
  

FOR d IN data
    INSERT d INTO caracteres
"""
# python-arango
aql.execute(insert_query)


<Cursor>

In [ ]:
listar_todos = """
FOR c IN caracteres
    RETURN c
"""
# python-arango
query_result = aql.execute(listar_todos)

# pyArango
# query_result = db.AQLQuery(all_characters_names, rawResults=True)
for doc in  query_result:
    print(doc)

{'_key': 'A', '_id': 'caracteres/A', '_rev': '_edCieX----', 'en': 'strong', 'es': 'fuerte'}
{'_key': 'B', '_id': 'caracteres/B', '_rev': '_edCieX---_', 'en': 'polite', 'es': 'polite'}
{'_key': 'C', '_id': 'caracteres/C', '_rev': '_edCieX---A', 'en': 'loyal', 'es': 'leal'}
{'_key': 'D', '_id': 'caracteres/D', '_rev': '_edCieX---B', 'en': 'beautiful', 'es': 'bonito'}
{'_key': 'E', '_id': 'caracteres/E', '_rev': '_edCieX---C', 'en': 'sneaky', 'es': 'insidioso'}
{'_key': 'F', '_id': 'caracteres/F', '_rev': '_edCieX---D', 'en': 'experienced', 'es': 'experimentado'}
{'_key': 'G', '_id': 'caracteres/G', '_rev': '_edCieX---E', 'en': 'corrupt', 'es': 'corrupto'}
{'_key': 'H', '_id': 'caracteres/H', '_rev': '_edCieX---F', 'en': 'powerful', 'es': 'potente'}
{'_key': 'I', '_id': 'caracteres/I', '_rev': '_edCieX---G', 'en': 'naive', 'es': 'ingenuo'}
{'_key': 'J', '_id': 'caracteres/J', '_rev': '_edCieX---H', 'en': 'unmarried', 'es': 'soltero'}
{'_key': 'K', '_id': 'caracteres/K', '_rev': '_edCieX--

In [ ]:
listar_personajes_caracteres = """
FOR c IN Personajes
    RETURN {nombre:c.nombre, caracteres:c.caracteres}
"""
query_result = aql.execute(listar_personajes_caracteres)
for doc in  query_result:
    print(doc)

{'nombre': 'Roberto', 'caracteres': ['A', 'H', 'C']}
{'nombre': 'Jaime', 'caracteres': ['A', 'F', 'B']}
{'nombre': 'Catalina', 'caracteres': ['D', 'H', 'C']}
{'nombre': 'Cesar', 'caracteres': ['H', 'E', 'F']}
{'nombre': 'David', 'caracteres': ['D', 'H', 'C']}
{'nombre': 'Jose', 'caracteres': ['A', 'B', 'C', 'F']}
{'nombre': 'Pedro', 'caracteres': ['E', 'G', 'F']}
{'nombre': 'Guillermo', 'caracteres': ['O', 'L', 'N']}
{'nombre': 'Juan', 'caracteres': ['A', 'B', 'C', 'F']}
{'nombre': 'Sara', 'caracteres': ['D', 'I', 'J']}
{'nombre': 'Ariela', 'caracteres': ['C', 'K', 'L']}
{'nombre': 'Roberto', 'caracteres': ['A', 'B', 'C', 'K']}
{'nombre': 'Theon', 'caracteres': ['E', 'R', 'K']}
{'nombre': 'Bran', 'caracteres': ['L', 'J']}
{'nombre': 'Joffrey', 'caracteres': ['I', 'L', 'O']}
{'nombre': 'Sandor', 'caracteres': ['A', 'P', 'K', 'F']}
{'nombre': 'Tyrion', 'caracteres': ['F', 'K', 'M', 'N']}
{'nombre': 'Khal', 'caracteres': ['A', 'C', 'O', 'P']}
{'nombre': 'Tywin', 'caracteres': ['O', 'M', '

# Ejemplos Joins

In [ ]:
#La función DOCUMENT() se puede utilizar para buscar uno o varios documentos a través de identificadores de documentos.
listar_personajes_caracteres = """
FOR c IN Personajes
    LIMIT 5
    RETURN {nombre:c.nombre, caracteres:DOCUMENT("caracteres", c.caracteres)}
"""
# python-arango
query_result = aql.execute(listar_personajes_caracteres)

for doc in  query_result:
    print(doc)

{'nombre': 'Roberto', 'caracteres': [{'_key': 'A', '_id': 'caracteres/A', '_rev': '_edCieX----', 'en': 'strong', 'es': 'fuerte'}, {'_key': 'H', '_id': 'caracteres/H', '_rev': '_edCieX---F', 'en': 'powerful', 'es': 'potente'}, {'_key': 'C', '_id': 'caracteres/C', '_rev': '_edCieX---A', 'en': 'loyal', 'es': 'leal'}]}
{'nombre': 'Jaime', 'caracteres': [{'_key': 'A', '_id': 'caracteres/A', '_rev': '_edCieX----', 'en': 'strong', 'es': 'fuerte'}, {'_key': 'F', '_id': 'caracteres/F', '_rev': '_edCieX---D', 'en': 'experienced', 'es': 'experimentado'}, {'_key': 'B', '_id': 'caracteres/B', '_rev': '_edCieX---_', 'en': 'polite', 'es': 'polite'}]}
{'nombre': 'Catalina', 'caracteres': [{'_key': 'D', '_id': 'caracteres/D', '_rev': '_edCieX---B', 'en': 'beautiful', 'es': 'bonito'}, {'_key': 'H', '_id': 'caracteres/H', '_rev': '_edCieX---F', 'en': 'powerful', 'es': 'potente'}, {'_key': 'C', '_id': 'caracteres/C', '_rev': '_edCieX---A', 'en': 'loyal', 'es': 'leal'}]}
{'nombre': 'Cesar', 'caracteres': [

In [ ]:
listar_personajes_caracteres = """
FOR c IN Personajes
    LIMIT 5
    RETURN {nombre:c.nombre, caracteres:DOCUMENT("caracteres", c.caracteres)[*].es}
"""
# python-arango
query_result = aql.execute(listar_personajes_caracteres)

for doc in  query_result:
    print(doc)

{'nombre': 'Roberto', 'caracteres': ['fuerte', 'potente', 'leal']}
{'nombre': 'Jaime', 'caracteres': ['fuerte', 'experimentado', 'polite']}
{'nombre': 'Catalina', 'caracteres': ['bonito', 'potente', 'leal']}
{'nombre': 'Cesar', 'caracteres': ['potente', 'insidioso', 'experimentado']}
{'nombre': 'David', 'caracteres': ['bonito', 'potente', 'leal']}


In [ ]:
listar_personajes_caracteres = """
FOR c IN Personajes
    LIMIT 5
    RETURN {nombre:c.nombre, caracteres:DOCUMENT("caracteres", c.caracteres)[*].en}
"""
# python-arango
query_result = aql.execute(listar_personajes_caracteres)

for doc in  query_result:
    print(doc)

{'nombre': 'Roberto', 'caracteres': ['strong', 'powerful', 'loyal']}
{'nombre': 'Jaime', 'caracteres': ['strong', 'experienced', 'polite']}
{'nombre': 'Catalina', 'caracteres': ['beautiful', 'powerful', 'loyal']}
{'nombre': 'Cesar', 'caracteres': ['powerful', 'sneaky', 'experienced']}
{'nombre': 'David', 'caracteres': ['beautiful', 'powerful', 'loyal']}


In [ ]:
todos_personajes_caracter = """
FOR c IN Personajes
    LIMIT 5
    RETURN MERGE(c, { caracter: DOCUMENT("caracteres", c.caracteres)[*].es } )
"""
query_result = aql.execute(todos_personajes_caracter)
for doc in  query_result:
    print(doc)

{'_id': 'Personajes/266302775424', '_key': '266302775424', '_rev': '_edBIuVq---', 'apellido': 'Ramirez', 'caracteres': ['A', 'H', 'C'], 'equipo': 'Colo Colo', 'nombre': 'Roberto', 'vivo': True, 'caracter': ['fuerte', 'potente', 'leal']}
{'_id': 'Personajes/266302775425', '_key': '266302775425', '_rev': '_edBIuVq--_', 'apellido': 'Quinteros', 'caracteres': ['A', 'F', 'B'], 'edad': 36, 'equipo': 'Colo Colo', 'nombre': 'Jaime', 'vivo': True, 'caracter': ['fuerte', 'experimentado', 'polite']}
{'_id': 'Personajes/266302775426', '_key': '266302775426', '_rev': '_edBIuVq--A', 'apellido': 'rojas', 'caracteres': ['D', 'H', 'C'], 'edad': 40, 'equipo': 'Colo Colo', 'nombre': 'Catalina', 'vivo': True, 'caracter': ['bonito', 'potente', 'leal']}
{'_id': 'Personajes/266302775427', '_key': '266302775427', '_rev': '_edBIuVq--B', 'apellido': 'Lanaro', 'caracteres': ['H', 'E', 'F'], 'edad': 36, 'equipo': 'Colo Colo', 'nombre': 'Cesar', 'vivo': True, 'caracter': ['potente', 'insidioso', 'experimentado']}


In [ ]:
all_characters_traits = """
FOR c IN Personajes
  LIMIT 10
  SORT c.apellido 
  RETURN MERGE(c, {
    caracter: (
      FOR key IN c.caracteres
        FOR t IN caracteres
          FILTER t._key == key
          RETURN t.es
    )
  })
"""
# python-arango
query_result = aql.execute(all_characters_traits)

for doc in  query_result:
    print(doc)

{'_id': 'Personajes/266302775430', '_key': '266302775430', '_rev': '_edBIuVq--E', 'apellido': 'Baesa', 'caracteres': ['E', 'G', 'F'], 'equipo': 'Colo Colo', 'nombre': 'Pedro', 'vivo': True, 'caracter': ['insidioso', 'corrupto', 'experimentado']}
{'_id': 'Personajes/266302775427', '_key': '266302775427', '_rev': '_edBIuVq--B', 'apellido': 'Lanaro', 'caracteres': ['H', 'E', 'F'], 'edad': 36, 'equipo': 'Colo Colo', 'nombre': 'Cesar', 'vivo': True, 'caracter': ['potente', 'insidioso', 'experimentado']}
{'_id': 'Personajes/266302775429', '_key': '266302775429', '_rev': '_edBIuVq--D', 'apellido': 'Mora', 'caracteres': ['A', 'B', 'C', 'F'], 'equipo': 'Colo Colo', 'nombre': 'Jose', 'vivo': True, 'caracter': ['fuerte', 'polite', 'leal', 'experimentado']}
{'_id': 'Personajes/266302775432', '_key': '266302775432', '_rev': '_edBIuVq--G', 'apellido': 'Niebla', 'caracteres': ['A', 'B', 'C', 'F'], 'edad': 16, 'equipo': 'Colo Colo', 'nombre': 'Juan', 'vivo': True, 'caracter': ['fuerte', 'polite', 'lea

* https://www.arangodb.com/docs/stable/aql/tutorial.html